# Exploring BFS

First, include some libraries

In [ ]:
# Library imports

#
# Import tensor class
#
from fibertree import Payload, Fiber, Tensor, TensorImage

#
# Import display classes/utilities
#
from IPython.display import display # to display images

def displayTensor(t):
    display(TensorImage(t).im)

#
# Helper for data directory
#
import os

data_dir = "../data"

def datafileName(filename):
    return os.path.join(data_dir, filename)



## Graph Inputs

In [ ]:
#
# Function to create graph inputs
#

def create_inputs(display=True):
    
    # Adjacency matrix - Ranks "S" (source) and "D" (destination)

    a = Tensor.fromUncompressed([ "S", "D"],
                                [ [ 0, 1, 1, 0, 0, 0 ],
                                  [ 0, 0, 1, 1, 0, 0 ],
                                  [ 0, 0, 0, 1, 1, 0 ],
                                  [ 0, 0, 0, 0, 1, 1 ],
                                  [ 1, 0, 0, 0, 0, 1 ],
                                  [ 1, 1, 0, 0, 0, 0 ] ])

    # Fringe (current) - Rank "V" (either source or destination)

    f0 = Tensor.fromUncompressed([ "V" ], [ 1, 0, 0, 0, 0, 0 ])

    # Distance - Rank "V" (either source or destination)

    d = Tensor.fromUncompressed([ "V" ], [1, 0, 0, 0, 0, 0])

    print("Adjacency Matrix")
    displayTensor(a)

    print("Distance Vector")
    displayTensor(d)

    print("Current Fringe")
    displayTensor(f0)

    return (a, f0, d)



# Naive BFS - source stationary (push)

This version traverses all neighbors of each source node, even if there already is a distance. So there is a check to not create a new distance.

In [ ]:
# Create inputs

(a, f0, d) = create_inputs()

# Setup for traversing roots

# Get root fibers
a_s = a.getRoot()
f0_s = f0.getRoot()
d_d = d.getRoot()



In [ ]:
level = 2

while (f0_s.countValues() > 0):
    print("\n\n")
    print(f"Level {level} fringe")
    displayTensor(f0_s.nonEmpty())
    print(f"Level {level} distances")
    displayTensor(d_d)   

    # Create a new next fringe (f1)
    
    f1 = Tensor(rank_ids=[ "V" ]) 
    f1_d = f1.getRoot()

    # For each source in fringe get destinations (neighbors)
    
    for s, (_, a_d) in f0_s & a_s:
        print(f"Processing source {s}")
        print(f"Neighbors:\n {a_d}")

        # For each neighboring destination 
        # prepare to update distance and next fringe
        
        for d, (f1_d_ref, (d_d_ref, _)) in f1_d << (d_d << a_d):
            print(f"  Processing destination {d} = {d_d_ref}")

            # Only update distance and fringe for "empty" destinations, 
            # i.e., without a distance (unvisited)
            
            if Payload.isEmpty(d_d_ref):
                print(f"  Adding destination {d}")

                # Update next fringe
                
                f1_d_ref += 1
                
                # Update destination's distance
                
                d_d_ref += level

    # Move to next level
    
    level += 1
    
    # Copy next fringe to current fringe
    
    f0 = f1
    f0_s = f0.getRoot()


print("\n\n")
print("Final Distances")
displayTensor(d_d)

## Optimized BFS - source stationary (push)

Avoid processing of any destination node that already has a distance by subtracting the distance array from the neighbors

In [ ]:
# Create inputs

(a, f0, d) = create_inputs()

# Setup for traversing roots

# Get root fibers
a_s = a.getRoot()
f0_s = f0.getRoot()
d_d = d.getRoot()



In [ ]:

level = 2


while (f0_s.countValues() > 0):
    print("\n\n")
    print(f"Level {level} fringe")
    displayTensor(f0_s.nonEmpty())
    print(f"Level {level} distances")
    displayTensor(d_d)   

    # Create a new next fringe (f1)
    
    f1 = Tensor(rank_ids=[ "D" ]) 
    f1_d = f1.getRoot()

    # For each source in fringe get destinations (neighbors)

    for s, (_, a_d) in f0_s & a_s:
        print(f"Processing source {s}")
        print(f"Neighbors:\n {a_d}")

        # For each neighboring destination without a distance
        # prepare to update distance and next fringe
            
        for d, (f1_d_ref, (d_d_ref, _)) in f1_d << (d_d << (a_d - d_d)):
            print(f"  Processing destination {d} = {d_d_ref}")
            print(f"  Adding destination {d}")

            # Update next frige (note no "if" statement)
            
            f1_d_ref += 1
            
            # Update destination's distance

            d_d_ref += level

    # Move to next level
    
    level += 1
    
    # Copy next fringe to current fringe
    
    f0 = f1
    f0_s = f0.getRoot()

print("\n\n")
print("Final Distances")
displayTensor(d_d)

### Destination stationary BFS (pull)

In [ ]:
# Create inputs

(a, f0, d) = create_inputs()

# Setup for traversing roots

# Get root fibers
a_s = a.getRoot()
f0_s = f0.getRoot()
d_d = d.getRoot()


# Transpose the adjacency matrix
at_d = a_s.swapRanks()

print("Transposed adjaceny matrix")
at = Tensor.fromFiber(["D", "S"], at_d)
displayTensor(at)



In [ ]:
# Destination Stationary

iteration = 1

while (f0_s.countValues() > 0):
    print("\n\n")
    print(f"Iteration {iteration} fringe")
    displayTensor(f0_s.nonEmpty())
    print(f"Iteration {iteration} distances")
    displayTensor(d_d)   

    # Create a new next fringe (f1)
    
    f1 = Tensor(rank_ids=[ "V" ]) 
    f1_d = f1.getRoot()
    

    # For destinations without a distance get incoming neighbors
    # and prepare for updates to distances and next fringe

    for d, (f1_d_ref, (d_d_ref, at_s)) in f1_d << (d_d << (at_d - d_d)):
        
        print(f"Processing destination {d}")
        print(f"Incoming neighbors:\n {at_s}")

        # For incoming sources in fringe with a distance
        # pick any source to assign a distance (we use the first)
        
        # Note, because all the sources are in the fringe they have the same distance!
        
        for s, ((_, _), d_s_val) in (at_s & f0_s) & d_d:
            print(f"  Processing source {s} = {d_s_val}")
            print(f"  Adding destination {d}")

            assert d_s_val != 0

            print(f"Debug fringe {f0_s!r}")
            # Update next fringe
            
            f1_d_ref += 1
            
            # Update destination's distance

            d_d_ref += d_s_val + 1
            break
            
    # Move to next iteration
    
    iteration += 1
    
    # Copy next fringe to current fringe
    
    f0 = f1
    f0_s = f0.getRoot()

print("\n\n")
print("Final Distances")
displayTensor(d_d)

## First source then destination stationary BFS - Push Pull

In [ ]:
# Create inputs

(a, f0, d) = create_inputs()

# Setup for traversing roots

# Get root fibers
a_s = a.getRoot()
f0_s = f0.getRoot()
d_d = d.getRoot()

In [ ]:
# Destination stationary (push) stage

level = 2


while (f0_s.countValues() > 0 and d_d.countValues() < 3):
    print("\n\n")
    print(f"Level {level} fringe")
    displayTensor(f0_s.nonEmpty())
    print(f"Level {level} distances")
    displayTensor(d_d)   

    # Create a new next fringe (f1)
    
    f1 = Tensor(rank_ids=[ "D" ]) 
    f1_d = f1.getRoot()

    # For each source in fringe get destinations (neighbors)

    for s, (_, a_d) in f0_s & a_s:
        print(f"Processing source {s}")
        print(f"Neighbors:\n {a_d}")

        # For each neighboring destination without a distance
        # prepare to update distance and next fringe
            
        for d, (f1_d_ref, (d_d_ref, _)) in f1_d << (d_d << (a_d - d_d)):
            print(f"  Processing destination {d} = {d_d_ref}")
            print(f"  Adding destination {d}")

            # Update next frige (note no "if" statement)
            
            f1_d_ref += 1
            
            # Update destination's distance

            d_d_ref += level

    # Move to next level
    
    level += 1
    
    # Copy next fringe to current fringe
    
    f0 = f1
    f0_s = f0.getRoot()

print("\n\n")  
print("Final destination stationary fringe")
displayTensor(f0)
print("Final destination stationary distances")
displayTensor(d_d)

In [ ]:
# Prepare for destination stations part

# Transpose the adjacency matrix
at_d = a_s.swapRanks()

print("Transposed adjaceny matrix")
at = Tensor.fromFiber(["D", "S"], at_d)
displayTensor(at)



In [ ]:
# Destination Stationary

iteration = 1

while (f0_s.countValues() > 0):
    print("\n\n")
    print(f"Iteration {iteration} fringe")
    displayTensor(f0_s.nonEmpty())
    print(f"Iteration {iteration} distances")
    displayTensor(d_d)   

    # Create a new next fringe (f1)
    
    f1 = Tensor(rank_ids=[ "V" ]) 
    f1_d = f1.getRoot()
    

    # For destinations without a distance get incoming neighbors
    # and prepare for updates to distances and next fringe

    for d, (f1_d_ref, (d_d_ref, at_s)) in f1_d << (d_d << (at_d - d_d)):
        
        print(f"Processing destination {d}")
        print(f"Incoming neighbors:\n {at_s}")

        # For incoming sources in fringe with a distance
        # pick any source to assign a distance (we use the first)
        
        # Note, because all the sources are in the fringe they have the same distance!

        xxx = at_s & f0_s
        print(f"Debug intersection: {xxx!r}")
        
        for s, ((_, _), d_s_val) in (at_s & f0_s) & d_d:
            print(f"  Processing source {s} = {d_s_val}")
            print(f"  Adding destination {d}")

            assert d_s_val != 0

            print(f"Debug fringe {f0_s!r}")
            # Update next fringe
            
            f1_d_ref += 1
            
            # Update destination's distance

            d_d_ref += d_s_val + 1
            break
            
    # Move to next iteration
    
    iteration += 1
    
    # Copy next fringe to current fringe
    
    f0 = f1
    f0_s = f0.getRoot()

print("\n\n")
print("Final Distances")
displayTensor(d_d)

## Testing area

For running alternative algorithms